# Introducción a Biopython

El proyecto de [Biopython](http://www.biopython.org) es una asociación internacional de desarrolladores del lenguaje [Python](https://www.python.org) con aplicación a biología molecular computacional. Es de libre acceso y está muy bien documentado, para mayor profundidad consulte el [Recetario de Biopython](http://biopython.org/DIST/docs/tutorial/Tutorial.html). Para la instalación y requisitos consultar https://github.com/biopython/biopython/blob/master/README.rst

# Instalación de Biopython en Apolo
Una introducción del servidor de EAFIT Apolo se encuentra [aquí](https://apolo-docs.readthedocs.io/en/latest/). 
Entre a Apolo por vía de putty o una terminal de linux (ya sea wsl o cygwin), previo inicio del Global Protect

In [ ]:
ssh lsalazarj@apolo.eafit.edu.co


Ahora vamos usar [conda](https://conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-python), un paquete que ayuda a gestionar las instalaciones con múltiples ambientes y versiones. Vamos a crear un ambiente ([environment](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#activating-an-environment)) llamado *biopython*, y allí vamos a instalar las librerías que necesitamos.

In [ ]:

module load python/3.10_miniconda-23.5.2
conda create -n biopython
conda activate biopython
conda install -c conda-forge biopython

Estos comandos instalan una versión por defecto que a veces no funciona. Para especificar la versión que uno quiere (por ejemplo biopython compatible con python3.9 en vez 3.11)

In [ ]:
conda create -n biopython_py3.9 python=3.9
conda activate biopython_py3.9
conda install -c conda-forge biopython

Debemos salir y volver a entrar a apolo para chequear la instalación. Una vez realizamos el login, activamos el ambiente de bioppython y comenzamos python

La siguiente vez que entran, ya instalado el ambiente: cargand el ambiente y lo activan

In [ ]:

module load python/3.10_miniconda-23.5.2
source activate biopython


In [ ]:

python

In [ ]:
import Bio
print(Bio.__version__)

Si sale un error después de llamar la instalación fue defectuosa. De lo contrario, está bien por el momento y empezamos a trabajar con secuencias. 

## Secuencias
Aunque una secuencia es un conjunto de letras, biopython tiene un formato especial para definir una secuencia

In [ ]:
from Bio.Seq import Seq
my_seq = Seq("AGTACACTGGT")
my_seq
my_seq.complement()
my_seq.reverse_complement()

Un recordatorio de la síntesis de proteínas

<img src="protein_synth.png" width=400 height=400 />

## Formato Fasta de Secuencias
Existen diferente tipos de formatos para almacenar datos biólogicos, según su naturaleza. Estos formatos son formas estandarizadas que permiten codificar la información de manera simplificada. Muchos de estos formatos son en texto plano, y por lo tanto se pueden visualizar en editores como Notepad++ (Windows) o Sublime Text (Mac). Para el almacenamiento de secuencias se utilizan principalmente dos tipos de formato: FASTA y FASTQ

<figure>
<img src="fasta.png" width=150 height=150 />

# Descargar y copiar archivos entre pc al servidor

Vamos a descargar la secuencias de DNA y la de proteínas de uno de los ensamblajes del virus del COVID-19 en formatos fasta

In [ ]:
wget http://ftp.ensemblgenomes.org/pub/viruses/fasta/sars_cov_2/dna/Sars_cov_2.ASM985889v3.dna_rm.toplevel.fa.gz 
wget http://ftp.ensemblgenomes.org/pub/viruses/fasta/sars_cov_2/pep/Sars_cov_2.ASM985889v3.pep.all.fa.gz

Para guardar una copia de estos archivos en su PC, puede utilizar WinSCP o abrir una terminal en el PC donde quiera guardar el archivo y darle el comando:

In [ ]:
scp lsalazarj@apolo.eafit.edu.co:/home/laura/clase_biopy/glycoproteins.py .

esto significa "save copy (scp) desde usuario@servidor:/folder/archivo aquí (.)" Ahora usted tiene copia de estos archivos en su PC. Ahora volvamos a Apolo

Descomprima los archivos para evaluar su contenido

In [ ]:
gunzip *.gz

el "*" indica que lo haga para todos los archivos que estan comprimidos (.gz)

In [ ]:
from Bio import SeqIO

dna_file = "Sars_cov_2.ASM985889v3.dna_rm.toplevel.fa"

for record in SeqIO.parse(dna_file, "fasta"):
    print(record)


La función SeqIO.parse abre el archivo **input_file** y lo intepreta como un archivo fasta. Como solo hay una secuencia, el for loop solo itera una vez y muestra ("print") los atributos de la secuencia. Compárelo con el archivo fasta para entender que significa cada línea.

Ahora vamos a extraer la secuencia y calcular su tamaño

In [ ]:
for record in SeqIO.parse(dna_file, "fasta"):
    print(record.seq)
    print(len(record.seq))


<div class="alert alert-block alert-warning">
<b>Nota:</b> La indentación (la sangría jerárquica luego de una declaración) es muy importante en Python. Si no se respeta, el programa arroja un error  </div>

De manera similar se hace con un archivo de múltiples secuencias, por ejemplo el archivo de proteínas.
Repita la copia de archivo como se hizo anteriormente, con el de las proteínas.
Ahora vamos a iterar sobre las secuencias del archivo "Sars_cov.pep.fa" y a desplegar los identificadores

In [ ]:
prot_file = "Sars_cov_2.ASM985889v3.pep.all.fa"

for record in SeqIO.parse(prot_file, "fasta"):
    print(record.id)

En este archivo hay 12 proteínas de diferentes tamaños y con funciones distintas. Vamos a imprimir su id, su tamaño y su descripción:

In [ ]:
for record in SeqIO.parse(prot_file, "fasta"):
    print(record.id,len(record.seq),record.description)

La descripción tiene mucha información importante pero esta no es muy clara, entonces vamos a extraer el fragmento que contiene la función, vamos a dividir la descripción en items, utilizando la función ``split.()`` y luego le damos el número (**index**) del item entre de interés ``[]`` (es el item 9 pero como python comienza a contar desde 0 entonces es el 8) 

In [ ]:
for record in SeqIO.parse(prot_file, "fasta"):
    print(record.id,len(record.seq),record.description.split()[8])

## Escribir secuencias un archivo  
Con frecuencia estamos interesados en un subconjunto de los genes o proteínas para realizar un análisis a profundidad de ellos. En este caso vamos a seleccionar las glicoproteínas que son aquellas que codifican la espícula. Esta espícula ("spike") es la encargada de mediar la entrada del virus hacia el hospedero y tiene una tasa de mutación mas alta que los otros genes, lo que presumiblemente dió origen a cepas mas virulentas (https://en.wikipedia.org/wiki/Coronavirus_spike_protein)

In [ ]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

for record in SeqIO.parse(prot_file, "fasta"):
    if (record.description.split()[9] == "glycoprotein"):
        print(record)

Hasta aquí hemos seleccionado las secuencias que contienen el término "glycoprotein". Para escribirlas en un archivo en un formato adecuado de secuencias vamos a utilizar la función de biopython  ``Bio.SeqIO.write()`` y a cambiar un poco el estilo. 

In [ ]:
glycoproteins = (record for record in SeqIO.parse(prot_file, "fasta") if (record.description.split()[9] == "glycoprotein"))
SeqIO.write(glycoproteins, "sars_covid19_glycoproteins.fa", "fasta")

Donde eperas que esté el archivo? Chequéalo mirando el directorio actual ("get current working directory"). Verifica que si se haya generado con el nombre y formatos correctos, y la información que deseamos que contenga. 

In [ ]:
import os

os.getcwd()
os.listdir()

Por último, vamos a escribir los comando que utilizamos en un script de python (con extensión ".py") para dejar registro de lo que hicimos y pueda ser repetible

In [ ]:
#!/usr/local/bin/

# Script por Laura Salazar Jaramillo
# Genera un subconjunto de glicoproteinas de SARS-covid 19

# Importamos las librerías necesarias
import os
import sys

import Bio
from Bio import SeqIO
from Bio.Seq import Seq

# Archivos de entrada (inputs)
dna_file = "/home/lsalazarj/bicomp2024/Sars_cov.dna.fa"
prot_file = "Sars_cov.prot.fa"


# Listamos el tamano del genoma
for record in SeqIO.parse(dna_file, "fasta"):
    print("La longitud en nc del genoma es:",len(record.seq))

# Listamos los id y longitudes de las proteínas
for record in SeqIO.parse(prot_file, "fasta"):
    print(record.id,len(record.seq),record.description.split()[9])

# Seleccionamos las glicoproteínas y las guardamos en un archivo aparte
glycoproteins = (record for record in SeqIO.parse(prot_file, "fasta") if (record.description.split()[9] == "glycoprotein"))
SeqIO.write(glycoproteins, "sars_covid19_glycoproteins.fa", "fasta")

Este script se puede ejecutar en la terminal escribiendo los comandos

In [ ]:
python glycoproteins.py > glycoproteins.out

Para copiar los archivos desde Apolo al computador personal entre a la terminal del computador y copie el archivo con la ruta completa desde Apolo al directorio donde está (indicaod por ".") o a otro directorio indicándole la ruta completa

In [ ]:
scp lsalazarj@apolo.eafit.edu.co:/home/laura/biopython/glycoproteins.py .

<div class="alert alert-block alert-info">
<b>Ejercicio</b> 
Ejecute en Apolo el script de python via Slurm 
</div>